# lru_cache предоставляет кэш последних результатов выполнения функций, или другими словами, запоминает результат их работы.

Делаем GET-запросы и кэшируем их результаты (до 32 результатов) с помощью декоратора @lru_cache

In [1]:
from functools import lru_cache
import requests

@lru_cache(maxsize=32)
def get_with_cache(url):
    try:
        r = requests.get(url)
        return r.text
    except:
        return "Not Found"


for url in ["https://google.com/",
            "https://martinheinz.dev/",
            "https://reddit.com/",
            "https://google.com/",
            "https://dev.to/martinheinz",
            "https://google.com/"]:
    get_with_cache(url)

print(get_with_cache.cache_info())
print(get_with_cache.cache_parameters())

CacheInfo(hits=2, misses=4, maxsize=32, currsize=4)
{'maxsize': 32, 'typed': False}


# cached_property - используется для кэширования результатов атрибутов класса.

In [2]:
import dataclasses
import functools
from typing import Optional
import hashlib


@dataclasses.dataclass
class User:
    first_name: str
    last_name: str
  
    @functools.cached_property
    def signature(self) -> bytes:
        return hashlib.sha512((self.first_name + self.last_name).encode()).digest()
    


# @total_ordering - декоратор сравнения

Нужно реализовать только метод eq и один из оставшихся методов, а остальные декоратор сгенерирует автоматически

In [4]:
from functools import total_ordering

@total_ordering
class Number:
 def __init__(self, value):
  self.value = value

 def __lt__(self, other):
  return self.value < other.value

 def __eq__(self, other):
  return self.value == other.value

print(Number(20) > Number(3))
print(Number(1) < Number(5))
print(Number(15) >= Number(15))
print(Number(10) <= Number(2))

True
True
True
False


# functools.wraps - декоратор для создания еще большего количества декораторов

Оборачиваем функцию, выполняющую определенную задачу (actual_func), внешним декоратором, и она сама становится декоратором, который затем можно применить к другим функциям, например, как в случае с greet.

In [6]:
def decorator(func):
    def actual_func(*args, **kwargs):
        print(f"Before Calling {func.__name__}")
        func(*args, **kwargs)
        print(f"After Calling {func.__name__}")

    return actual_func

@decorator
def greet(name):
    """Says hello to somebody"""
    print(f"Hello, {name}!")

greet("Martin")

Before Calling greet
Hello, Martin!
After Calling greet


# reduce - берет итерируемый объект и сворачивает (складывает) все его значения в одно

In [7]:
from functools import reduce
import operator

def product(iterable):
    return reduce(operator.mul, iterable, 1)

def factorial(n):
    return reduce(operator.mul, range(1, n))

def sum(numbers):  # Use `sum` function from standard library instead
    return reduce(operator.add, numbers, 1)

def reverse(iterable):
    return reduce(lambda x, y: y+x, iterable)

print(product([1, 2, 3]))
print(factorial(5))
print(sum([2, 6, 8, 3]))
print(reverse("hello"))

6
24
20
olleh


# singledispatchmethod() модуля functools создает из обычного метода класса - универсальный метод одиночной диспетчеризации

In [10]:
from functools import singledispatchmethod


class Negator:
    @singledispatchmethod
    def neg(self, arg):
        raise NotImplementedError("Cannot negate a")

    @neg.register
    def _(self, arg: int):
        return -arg

    @neg.register
    def _(self, arg: bool):
        return not arg